In [1]:
import os
import posixpath
import xarray as xr
import rioxarray
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt

import geometry.internal_orientation as io
import geometry.external_orientation as eo
import warnings

warnings.filterwarnings("ignore")

# Orthorectification tests

The purpose of this notebook is to build and test the functions necessary to orthorectify KH5-ARGON images. It is divided in two sections: internal orientation (IO), for the link between fiducial and image coordinate systems, and external orientation (EO), for the link between geocentric caresian, local cartesian and WGS84 coordinate systems, and the retrieval of satellite location and attitude.

# Internal Orientation (IO)

## Tests of non-linear transformations:
(xc, yc, alpha, delta_xi, delta_eta)

In [2]:
x, y = 1, 12
xc, yc = 0.33, 25
alpha_t = 0 * np.pi/180
delta_xi, delta_eta = 0.1, 22.3
xi, eta = io.image_to_fiducial_coordinates_molnar(x, y, xc, yc, alpha_t, delta_xi, delta_eta)
x1, y1 = io.fiducial_to_image_coordinates_molnar(xi, eta, xc, yc, alpha_t, delta_xi, delta_eta)
x1, x, y1, y

(1.0, 1, 12.0, 12)

In [3]:
xc_t, yc_t = 1, 1
alpha_t = 30 * np.pi/180
delta_xi_t, delta_eta_t = 1, 1

FMs_fiducial_coords = np.array([
    [0, 0],
    [1, 0],
    [0, 1],
    [1, 1]
]) * 12

FMs_image_coords = np.array(
    [np.array(io.fiducial_to_image_coordinates_molnar(XI[0], XI[1], xc_t, yc_t, alpha_t, delta_xi_t, delta_eta_t)) for XI in FMs_fiducial_coords]
)

res = opt.least_squares(
    io.objective_function_molnar,
    x0=[0, 0, 0, 1, 1],
    args=(FMs_fiducial_coords, FMs_image_coords),
)
params = res.x

xc, yc, alpha, delta_xi, delta_eta = params[0], params[1], params[2], params[3], params[4]
x, y = 1, 1
xi_t, eta_t = io.image_to_fiducial_coordinates_molnar(x, y, xc_t, yc_t, alpha_t, delta_xi_t, delta_eta_t)
xi, eta = io.image_to_fiducial_coordinates_molnar(x, y, xc, yc, alpha, delta_xi, delta_eta)
xi - xi_t, eta - eta_t

(0.08936108044652447, 0.08929586864952085)

In [4]:
xc_t, yc_t = 1, 1
alpha_t = 30 * np.pi/180
delta_xi_t, delta_eta_t = 1, 1

FMs_fiducial_coords = np.array([
    [0, 0],
    [1, 0],
    [0, 1],
    [1, 1]
]) * 12

FMs_image_coords = np.array(
    [np.array(io.fiducial_to_image_coordinates_molnar(XI[0], XI[1], xc_t, yc_t, alpha_t, delta_xi_t, delta_eta_t)) for XI in FMs_fiducial_coords]
)

res = opt.minimize(
    io.objective_function_molnar,
    x0=[0, 0, 0, 1, 1],
    args=(FMs_fiducial_coords, FMs_image_coords),
    method='BFGS',
)
if res.success == False:
    print(res)
params = res.x

xc, yc, alpha, delta_xi, delta_eta = params[0], params[1], params[2], params[3], params[4]
x, y = 1, 1
xi_t, eta_t = io.image_to_fiducial_coordinates_molnar(x, y, xc_t, yc_t, alpha_t, delta_xi_t, delta_eta_t)
xi, eta = io.image_to_fiducial_coordinates_molnar(x, y, xc, yc, alpha, delta_xi, delta_eta)
xi - xi_t, eta - eta_t

(1.90824833389579e-07, 5.521280728280169e-08)

## Linear transformation:
matrix: np.array([r11, r12, r21, r22])

In [5]:
x, y = 0.3, 1

alpha = 30 * np.pi/180
matrix = np.array([
    [12, 0],
    [0, 12]
]) @ np.array([
    [np.cos(alpha), np.sin(alpha)],
    [-np.sin(alpha), np.cos(alpha)]
])
print(matrix)

xi, eta = io.image_to_fiducial_coordinates_linear(x, y, matrix)
x1, y1 = io.fiducial_to_image_coordinates_linear(xi, eta, matrix)
x1, x, y1, y

[[10.39230485  6.        ]
 [-6.         10.39230485]]


(0.30000000000000004, 0.3, 1.0, 1)

In [6]:
alpha_t = 30 * np.pi/180
matrix_t = np.array([
    [12, 0],
    [0, 12]
]) @ np.array([
    [np.cos(alpha_t), np.sin(alpha_t)],
    [-np.sin(alpha_t), np.cos(alpha_t)]
])
print(matrix_t)

FMs_fiducial_coords = np.array([
    [0, 0],
    [1, 0],
    [0, 1],
    [1, 1]
]) * 12

FMs_image_coords = np.array(
    [np.array(io.fiducial_to_image_coordinates_linear(XI[0], XI[1], matrix-y)) for XI in FMs_fiducial_coords]
)

res = opt.least_squares(
    io.objective_function_linear,
    x0=[1, 0, 0, 1],
    args=(FMs_fiducial_coords, FMs_image_coords),
)
matrix = res.x

matrix = np.array([
    [matrix[0], matrix[1]],
    [matrix[2], matrix[3]]
])
print(matrix)
x, y = 1, 1
xi_t, eta_t = io.image_to_fiducial_coordinates_linear(x, y, matrix_t)
xi, eta = io.image_to_fiducial_coordinates_linear(x, y, matrix)
xi - xi_t, eta - eta_t

[[10.39230485  6.        ]
 [-6.         10.39230485]]
[[ 9.3921358   5.00093505]
 [-6.99922332  9.39280739]]


(-1.9992340023282615, -1.998720772797161)

In [7]:
alpha_t = 30 * np.pi/180
matrix_t = np.array([
    [12, 0],
    [0, 12]
]) @ np.array([
    [np.cos(alpha_t), np.sin(alpha_t)],
    [-np.sin(alpha_t), np.cos(alpha_t)]
])
print(matrix_t)

FMs_fiducial_coords = np.array([
    [0, 0],
    [1, 0],
    [0, 1],
    [1, 1]
]) * 12

FMs_image_coords = np.array(
    [np.array(io.fiducial_to_image_coordinates_linear(XI[0], XI[1], matrix_t)) for XI in FMs_fiducial_coords]
)

matrix = io.least_squares_linear(FMs_fiducial_coords, FMs_image_coords, alpha = 0.1, a_priori=np.array([[12, 0], [0, 12]]))
print(matrix)

x, y = 1, 1
xi_t, eta_t = io.image_to_fiducial_coordinates_linear(x, y, matrix_t)
xi, eta = io.image_to_fiducial_coordinates_linear(x, y, matrix)
xi - xi_t, eta - eta_t

[[10.39230485  6.        ]
 [-6.         10.39230485]]
[[10.62011906  5.75930513]
 [-5.50169194 10.36250587]]


(-0.012880659370718917, 0.46850908576633987)

In [8]:
x, y = 0.3, 1

alpha = 30 * np.pi/180
matrix = np.array([
    [1.0, 0],
    [0, 1.5]
]) @ np.array([
    [np.cos(alpha), np.sin(alpha)],
    [-np.sin(alpha), np.cos(alpha)]
])
print(matrix)
translation_vector = np.array([1, 2])
print(translation_vector)

xi, eta = io.image_to_fiducial_coordinates_affine(x, y, matrix, translation_vector)
x1, y1 = io.fiducial_to_image_coordinates_affine(xi, eta, matrix, translation_vector)
x1 - x, y1 - y

[[ 0.8660254   0.5       ]
 [-0.75        1.29903811]]
[1 2]


(-1.6653345369377348e-16, 0.0)

In [9]:
alpha = 30 * np.pi/180
matrix = np.array([
    [1.0, 0],
    [0, 1.5]
]) @ np.array([
    [np.cos(alpha), np.sin(alpha)],
    [-np.sin(alpha), np.cos(alpha)]
])
print(matrix)
translation_vector_t = np.array([1, 2])
print(translation_vector_t)

FMs_fiducial_coords = np.array([
    [0, 0],
    [1, 0],
    [0, 1],
    [1, 1]
]) * 12

FMs_image_coords = np.array(
    [np.array(io.fiducial_to_image_coordinates_affine(XI[0], XI[1], matrix_t, translation_vector_t)) for XI in FMs_fiducial_coords]
)

res = opt.minimize(
    io.objective_function_affine,
    x0=[1, 0, 0, 1, 0, 0],
    args=(FMs_fiducial_coords, FMs_image_coords),
    method="BFGS"
)
matrix = np.array([
    [res.x[0], res.x[1]],
    [res.x[2], res.x[3]]
])
translation_vector = np.array([[res.x[4]], [res.x[5]]])

x, y = 1, 1
xi_t, eta_t = io.image_to_fiducial_coordinates_affine(x, y, matrix_t, translation_vector_t)
xi, eta = io.image_to_fiducial_coordinates_affine(x, y, matrix, translation_vector)
xi - xi_t, eta - eta_t

[[ 0.8660254   0.5       ]
 [-0.75        1.29903811]]
[1 2]


(-9.417878032991212e-09, -2.419475464421339e-08)

# External orientation (EO)

In [ ]:
lat, lon, h = 78, 15, 0
x, y, z = eo.ellipsoidal_to_geocentric_cartesian_coordinates(lat * np.pi/180, lon * np.pi/180, h)
lat1, lon1, h1 = eo.geocentric_cartesian_to_ellipsoidal_coordinates(x, y, z)
lat1 * 180/np.pi - lat, lon1 * 180/np.pi - lon, h1 - h

In [ ]:
lat, lon, h = 78, 15, 0
x, y, z = eo.ellipsoidal_to_geocentric_cartesian_coordinates(lat * np.pi/180, lon * np.pi/180, h)

lat_c, lon_c = 75, 10
x_gr, y_gr, z_gr = eo.geocentric_cartesian_to_local_cartesian_coordinates(x, y, z, lat_c * np.pi/180, lon_c * np.pi/180)
x1, y1, z1 = eo.local_cartesian_to_geocentric_cartesian_coordinates(x_gr, y_gr, z_gr, lat_c * np.pi/180, lon_c * np.pi/180)
x1 - x, y1 - y, z1 - z

In [ ]:
# space rejection